In [91]:
import pandas as pd
import datetime as dt

In [92]:
f = lambda s: dt.datetime.strptime(s,'%d-%m-%Y %H:%M:%S')
# df_bn5 = pd.read_csv("data/historical/BankNiftyFut-Sep-5min-20200908.csv", sep="\t", parse_dates=['Date Time'], date_parser=f, encoding="utf_16_LE") 
df_bn5 = pd.read_csv("data/historical/NiftyFut-Sep-5min-20200911.csv", sep="\t", parse_dates=['Date Time'], encoding="utf_16_LE") 

#### Remove last empty column and set date time as index

In [93]:
df_bn5 = df_bn5[df_bn5.columns[:-1]]
df_bn5.set_index('Date Time', inplace=True)

## Input the evaluation date below

In [94]:
eval_date = "2020-09-11"

#### Extract a separate dataframe for the evaluation date

In [95]:
today_df_bn5 = df_bn5[eval_date]

#### Apply rolling mean columns for data

In [96]:
today_df_bn5['SMA6'] = today_df_bn5['Close'].rolling(6).mean()
today_df_bn5['SMA21'] = today_df_bn5['Close'].rolling(21).mean()

<ipython-input-96-f2f66b587a2f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  today_df_bn5['SMA6'] = today_df_bn5['Close'].rolling(6).mean()
<ipython-input-96-f2f66b587a2f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  today_df_bn5['SMA21'] = today_df_bn5['Close'].rolling(21).mean()


In [97]:
today_df_bn5.head(5)

,Open,Low,High,Close,Volume,SMA6,SMA21
Date Time,,,,,,,
2020-09-11 09:15:00,11430.40,11422.00,11463.00,11460.30,458102,NaN,NaN
2020-09-11 09:20:00,11465.00,11461.00,11477.85,11473.50,340425,NaN,NaN
2020-09-11 09:25:00,11474.00,11470.05,11487.40,11484.00,264825,NaN,NaN
2020-09-11 09:30:00,11485.00,11470.10,11486.00,11474.00,276750,NaN,NaN
2020-09-11 09:35:00,11473.35,11453.95,11473.95,11455.85,253125,NaN,NaN


### Strategy function

In [98]:
def backtest_Intraday_5min_strategy(df):
    SELL = False
    BUY = False
    SELL_PRICE = 0
    BUY_PRICE = 0
    GAIN_POINTS = 0
    SELL_STOPLOSS = 0
    BUY_STOPLOSS = 0
    STOPLOSS = 25
    SMA6_dead_band = 10
    Total_Signals = 0
    Total_Sell_Signal = 0
    Total_Buy_Signal = 0
    start_time = dt.datetime.strptime("10:00", "%H:%M").time()
    close_time = dt.datetime.strptime("15:15", "%H:%M").time()
    entry_close_time = dt.datetime.strptime("15:00", "%H:%M").time()
    print("Time\t\t\t", "Close\t\t", "SMA6\t\t", "SMA21\t\t", "SIGNAL\t", "TYPE\t\t",  "Points\t",)
    
    for row in df.itertuples():
        SMA6_upper_band = row.SMA6 + SMA6_dead_band
        SMA6_lower_band = row.SMA6 - SMA6_dead_band
        
        # SELL STOPLOSS
        if row.Index.time() >= start_time and row.Index.time() <= close_time: 
            points = 0
            if SELL and row.Close > SELL_STOPLOSS:
                SELL = False
                BUY = False
                Total_Signals += 1
                Total_Buy_Signal += 1
                points = SELL_PRICE - row.Close
                GAIN_POINTS = GAIN_POINTS + points
                SELL_PRICE = 0
                print(row.Index, "\t",  row.Close, "\t", round(row.SMA6,2),  "\t", round(row.SMA21,2),  "\t", "BUY\t", "SELL STOPLOSS\t", round(points,2))
            
            # BUY STOPLOSS
            points = 0
            if BUY and row.Close < BUY_STOPLOSS:
                points = row.Close - BUY_PRICE
                GAIN_POINTS = GAIN_POINTS + points
                BUY_PRICE = 0
                SELL = False
                BUY = False
                Total_Signals += 1
                Total_Sell_Signal += 1
                print(row.Index, "\t",  row.Close, "\t", round(row.SMA6,2),  "\t", round(row.SMA21,2),  "\t", "SELL\t", "BUY STOPLOSS\t", round(points,2))
            
            # SELL ENTRY
            points = 0
            if row.Close < SMA6_lower_band and \
                    row.Index.time() < entry_close_time and \
                    SELL is False:
                SELL_PRICE = row.Close
                SELL_STOPLOSS = SELL_PRICE + STOPLOSS
                if BUY_PRICE > 0:
                    points = SELL_PRICE - BUY_PRICE
                    GAIN_POINTS = GAIN_POINTS + points
                SELL = True
                BUY = False
                Total_Sell_Signal += 1
                Total_Signals += 1
                print(row.Index, "\t",  row.Close, "\t", round(row.SMA6,2),  "\t", round(row.SMA21,2),  "\t", "SELL\t", "SELL ENTRY\t", round(points,2))
             
            # BUY ENTRY
            points = 0
            if row.Close > SMA6_upper_band and \
                    row.Index.time() < entry_close_time and \
                    BUY is False:
                BUY_PRICE = row.Close
                BUY_STOPLOSS = BUY_PRICE - STOPLOSS
                if SELL_PRICE > 0:
                    points = SELL_PRICE - BUY_PRICE
                    GAIN_POINTS = GAIN_POINTS + points
                BUY = True
                SELL = False
                Total_Buy_Signal += 1
                Total_Signals += 1
                print(row.Index, "\t", row.Close,  "\t", round(row.SMA6,2),  "\t", round(row.SMA21,2),  "\t", "BUY\t", "BUY ENTRY\t", round(points,2))
            
            # DAY Close
            points = 0    
            if row.Index.time() >= close_time:
                if SELL:
                    SELL = False
                    BUY = False
                    Total_Signals += 1
                    Total_Buy_Signal += 1
                    points = SELL_PRICE - row.Close
                    GAIN_POINTS = GAIN_POINTS + points
                    SELL_PRICE = 0
                    print(row.Index, "\t",  row.Close, "\t", round(row.SMA6,2),  "\t", round(row.SMA21,2),  "\t", "BUY\t", "BUY DAYCLOSE\t", round(points,2))

                if BUY:
                    SELL = False
                    BUY = False
                    Total_Signals += 1
                    Total_Buy_Signal += 1
                    points = row.Close - BUY_PRICE
                    GAIN_POINTS = GAIN_POINTS + points
                    SELL_PRICE = 0
                    print(row.Index, "\t",  row.Close, "\t", round(row.SMA6,2),  "\t", round(row.SMA21,2),  "\t", "SELL\t", "SELL DAYCLOSE\t", round(points,2))
                    
                    
    
    print("\n\nTotal Signals:\t\t", Total_Signals)
    print("Total Sell Signals:\t", Total_Buy_Signal)
    print("Total Buy Signals:\t", Total_Sell_Signal)
    print("Total Points Gained:\t", round(GAIN_POINTS,2))

In [99]:
backtest_Intraday_5min_strategy(today_df_bn5)

Time			 Close		 SMA6		 SMA21		 SIGNAL	 TYPE		 Points	
2020-09-11 10:55:00 	 11475.75 	 11456.92 	 11454.73 	 BUY	 BUY ENTRY	 0
2020-09-11 11:30:02 	 11449.0 	 11456.75 	 11453.12 	 SELL	 BUY STOPLOSS	 -26.75
2020-09-11 12:10:00 	 11458.0 	 11446.07 	 11453.12 	 BUY	 BUY ENTRY	 0
2020-09-11 13:05:07 	 11443.55 	 11458.57 	 11452.5 	 SELL	 SELL ENTRY	 -14.45
2020-09-11 14:05:00 	 11454.0 	 11443.0 	 11446.28 	 BUY	 BUY ENTRY	 -10.45


Total Signals:		 5
Total Sell Signals:	 3
Total Buy Signals:	 2
Total Points Gained:	 -51.65
